In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/home/vice/DEZ/amazon_product_review/key/de-amazon-product-review-88dd24afade6.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/vice/DEZ/spark/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/04/22 16:40:38 WARN Utils: Your hostname, peace resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/22 16:40:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/22 16:40:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [33]:
df = spark.read.parquet('gs://de-amazon-product-review-bucket/staging/item/')


In [7]:
df.schema

StructType([StructField('main_category', StringType(), True), StructField('parent_asin', StringType(), True), StructField('average_rating', DoubleType(), True), StructField('rating_number', DoubleType(), True), StructField('price', StringType(), True), StructField('categories', ArrayType(ArrayType(StringType(), True), True), True), StructField('title', StringType(), True), StructField('store', StringType(), True)])

In [34]:
df.count()

35393189

In [15]:
df.filter(df['main_category']=='All Beauty').select('main_category').distinct().show()

[Stage 11:======================================================> (33 + 1) / 34]

+-------------+
|main_category|
+-------------+
|   All Beauty|
+-------------+



In [35]:
from pyspark.sql.functions import col, size
df.filter(
    col("categories").isNotNull() & (size(col("categories")) > 0)
).show()

[Stage 53:===================================================>      (8 + 1) / 9]

+-------------+-----------+--------------+-------------+-----+----------+-----+-----+
|main_category|parent_asin|average_rating|rating_number|price|categories|title|store|
+-------------+-----------+--------------+-------------+-----+----------+-----+-----+
+-------------+-----------+--------------+-------------+-----+----------+-----+-----+

